## Rod Cantilever Example

In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import numpy as np

import dismech

b = 0.02
h = 0.001

geom = dismech.GeomParams(rod_r0=0.001,
                          shell_h=0,
                          axs=b*h,
                          ixs1=b*h**3/12,
                          ixs2=h*b**3/12,
                          jxs=b*h**3/6)

material = dismech.Material(density=1200,
                            youngs_rod=2e6,
                            youngs_shell=0,
                            poisson_rod=0.5,
                            poisson_shell=0)

static_2d_sim = dismech.SimParams(static_sim=False,
                                  two_d_sim=True,   # no twisting
                                  use_mid_edge=False,
                                  use_line_search=False,
                                  show_floor=False,
                                  log_data=True,
                                  log_step=1,
                                  dt=1e-2,
                                  max_iter=25,
                                  total_time=1.0,
                                  plot_step=1,
                                  tol=1e-4,
                                  ftol=1e-4,
                                  dtol=1e-2)

env = dismech.Environment()
#env.add_force('gravity', g=np.array([0.0, 0.0, -9.81]))
env.add_force('selfContact', delta=0.2, h=0.02)
#env.set_static()

geo = dismech.Geometry.from_txt('input.txt')

robot = dismech.SoftRobot(geom, material, geo, static_2d_sim, env)

u = robot.state.u
u[8] = -0.1
u[11] = -0.1

### Time Stepping

As we are performing a static simulation, we must change gravity for each time step.

In [17]:
stepper = dismech.ImplicitEulerTimeStepper(robot)

robots = stepper.simulate()

qs = np.stack([robot.state.q for robot in robots])

[0.00000000e+00 0.00000000e+00 1.61841252e-06 0.00000000e+00
 0.00000000e+00 1.61841252e-06 0.00000000e+00 0.00000000e+00
 1.19998382e-01 0.00000000e+00 0.00000000e+00 1.19998382e-01
 0.00000000e+00 0.00000000e+00] [[ 1.60000000e+02  0.00000000e+00  1.61841252e-06 -4.00000000e+01
   0.00000000e+00 -1.61841252e-06  0.00000000e+00 -8.98399420e-23
   0.00000000e+00  0.00000000e+00  8.98399420e-23  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.20000000e+02  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   1.61841252e-06  0.00000000e+00  0.00000000e+00 -1.61841252e-06
   0.00000000e+00  0.00000000e+00]
 [ 1.61841252e-06  0.00000000e+00  1.20000121e+02  1.61841252e-06
   0.00000000e+00  1.20056438e-04 -1.61841252e-06  0.00000000e+00
  -1.20380121e-04 -1.61841252e-06  0.00000000e+00 -1.20380121e-04
   0.00000000e+00  0.00000000e+00]
 [-4.00000000e+01  0.00000000e+00  1.61841252e-06  1.60000000e+02
   0.00000000e+00 -1

In [18]:
t = np.arange(robot.sim_params.total_time, step=robot.sim_params.dt)
options = dismech.AnimationOptions(title='Rod Cantilever (N=51)')

fig = dismech.get_interactive_animation_plotly(robot, t, qs, options)
fig.show()